In [1]:
from pathlib import Path
from fishsense_data_processing_workflow_worker.config import settings
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from datetime import datetime
from fishsense_api_sdk.models.laser_label import LaserLabel
from tqdm.notebook import tqdm
from copy import deepcopy
import cv2
import asyncio

In [2]:
DIVE_ID = 427
PROJECT_ID = 73

In [3]:
OUTPUT_FOLDER = (Path("../output") / "preprocess_jpeg").absolute()

OUTPUT_FOLDER.exists()

True

In [4]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [5]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [6]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    dive = await fs.dives.get(dive_id=DIVE_ID)

dive

Dive(id=427, name='082124_Alligator Lighthouse drift_FSL06', path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06', dive_datetime=datetime.datetime(2024, 8, 21, 9, 55, 29, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=6, dive_slate_id=None)

In [7]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    images = await fs.images.get(dive_id=dive.id)

len(images), images[0]

(242,
 Image(id=118610, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210002.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 8, 56, 51, tzinfo=TzInfo(0)), checksum='0b7cd4da72d54172f1f9daf40ce4047f', is_canonical=True, dive_id=427, camera_id=6))

In [8]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    existing_laser_labels = await client.labels.get_laser_labels(dive.id)

len(existing_laser_labels), existing_laser_labels

(240,
 [LaserLabel(id=60353, label_studio_task_id=160111, label_studio_project_id=42, x=2024.4172874479698, y=1140.7199851362793, label='Green Laser', updated_at=datetime.datetime(2025, 5, 8, 23, 35, 39, 107201, tzinfo=TzInfo(0)), superseded=False, completed=True, label_studio_json={'allow_skip': True, 'annotations': [{'id': 88227, 'result': [{'id': 'Q5rOel__iQ', 'type': 'keypointlabels', 'value': {'x': 50.433913488987784, 'y': 37.82228067427982, 'width': 0.1286592815556511, 'keypointlabels': ['Green Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}], 'created_username': 'Andy Prochaska ajpro@outlook.com, 33', 'created_ago': '8\xa0months, 2\xa0weeks', 'completed_by': 33, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-05-08T23:35:38.962058Z', 'updated_at': '2025-05-08T23:35:38.962080Z', 'draft_created_at': None, 'lead_time': 9.075, 'import_id': None, 'last_action': None, 'bulk_

In [9]:
laser_labels_by_image_id = {label.image_id: label for label in existing_laser_labels}

len(laser_labels_by_image_id), laser_labels_by_image_id

(240,
 {118824: LaserLabel(id=60353, label_studio_task_id=160111, label_studio_project_id=42, x=2024.4172874479698, y=1140.7199851362793, label='Green Laser', updated_at=datetime.datetime(2025, 5, 8, 23, 35, 39, 107201, tzinfo=TzInfo(0)), superseded=False, completed=True, label_studio_json={'allow_skip': True, 'annotations': [{'id': 88227, 'result': [{'id': 'Q5rOel__iQ', 'type': 'keypointlabels', 'value': {'x': 50.433913488987784, 'y': 37.82228067427982, 'width': 0.1286592815556511, 'keypointlabels': ['Green Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}], 'created_username': 'Andy Prochaska ajpro@outlook.com, 33', 'created_ago': '8\xa0months, 2\xa0weeks', 'completed_by': 33, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-05-08T23:35:38.962058Z', 'updated_at': '2025-05-08T23:35:38.962080Z', 'draft_created_at': None, 'lead_time': 9.075, 'import_id': None, 'last_action': None

In [10]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    dive_images = await client.images.get(dive_id=DIVE_ID)

len(dive_images), dive_images

(242,
 [Image(id=118610, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210002.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 8, 56, 51, tzinfo=TzInfo(0)), checksum='0b7cd4da72d54172f1f9daf40ce4047f', is_canonical=True, dive_id=427, camera_id=6),
  Image(id=118611, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210001.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 8, 56, 51, tzinfo=TzInfo(0)), checksum='45dc5a454b35601b9dafabf24822195d', is_canonical=True, dive_id=427, camera_id=6),
  Image(id=118612, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210003.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 8, 56, 51, tzinfo=TzInfo(0)), checksum='2bf7c535f1b19de2ba3871d5aa8a240d', is_canonical=True, dive_id=427, camera_id=6),
  Image(id=118613, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/

In [11]:
incomplete_images = [image for image in dive_images if laser_labels_by_image_id.get(image.id) is None or not laser_labels_by_image_id[image.id].completed]

len(incomplete_images), incomplete_images

(2,
 [Image(id=118639, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210032.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 9, 1, 50, tzinfo=TzInfo(0)), checksum='b64d1f35673ccda7b1774717ec8174a3', is_canonical=True, dive_id=427, camera_id=6),
  Image(id=118786, path='2025-02-10 REEF Data Dump SMILE 6/082124_Alligator_FSL06/082124_Alligator Lighthouse drift_FSL06/P8210177.ORF', taken_datetime=datetime.datetime(2024, 8, 21, 9, 36, tzinfo=TzInfo(0)), checksum='6f090ace2544f68be9a20290bd2a5fae', is_canonical=True, dive_id=427, camera_id=6)])

In [12]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

now

'2026-01-26 18:50:52'

In [13]:
tasks = []
template = {
    "model_version": f"sql_labels_{now}",
    "result": []
}

for image in tqdm(incomplete_images):
    image_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

    img = cv2.imread(str(image_path))
    height, width, _ = img.shape

    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/{image.checksum}"
        },
        "annotations": []
    })

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
tasks

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/b64d1f35673ccda7b1774717ec8174a3'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/6f090ace2544f68be9a20290bd2a5fae'},
  'annotations': []}]

In [15]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ImportTasksProjectsResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=0.13872337341308594, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=2, prediction_count=0, task_ids=[225009, 225010])

In [16]:
laser_labels = [LaserLabel(id=None, image_id=image.id, label_studio_task_id=task_id, label_studio_project_id=PROJECT_ID, image_url=f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/{image.checksum}", updated_at=None, completed=False, label_studio_json={}, user_id=None, superseded=False, x=None, y=None, label=None) for task_id, image in zip(imported_tasks.task_ids, incomplete_images)]

len(laser_labels), laser_labels

(2,
 [LaserLabel(id=None, label_studio_task_id=225009, label_studio_project_id=73, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=118639, user_id=None),
  LaserLabel(id=None, label_studio_task_id=225010, label_studio_project_id=73, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=118786, user_id=None)])

In [17]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    async with asyncio.TaskGroup() as tg:
        for laser_label in tqdm(laser_labels):
            tg.create_task(fs.labels.put_laser_label(laser_label.image_id, laser_label))

  0%|          | 0/2 [00:00<?, ?it/s]